# Bag-of-Objects TF-IDF

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pickle
import pandas as pd
from tqdm import tqdm
import os

from dataloader import he_sampling
from data import FlickrDataset, CocoDataset, VGDataset
from PIL import Image
import matplotlib.patches as patches

from sklearn.feature_extraction.text import TfidfTransformer


In [3]:
# ds = VGDataset(sg_path=None)
# ds = FlickrDataset(sg_path=None)
ds = CocoDataset(sg_path=None)
# ds = VGDataset(sg_path='/data/project/rw/CBIR/data/vg_coco/vg_coco_gt_sg.pkl')

test: 5000
restval: 30504
val: 5000
train: 82783
loading annotations into memory...
Done (t=0.81s)
creating index...
index created!
loading annotations into memory...
Done (t=0.41s)
creating index...
index created!


In [4]:
ds.sg[0].keys()

dict_keys(['node_labels', 'adj', 'cocoid', 'imgid', 'bboxes'])

In [6]:
'''visual genome'''
# l_train_terms = []
# for img_id in ds.d_split['train']:
#     sg = ds.imgid2sg(img_id)
#     l_train_terms.append(sg['obj_labels'])
    
# l_test_terms = []
# for img_id in ds.d_split['test']:
#     sg = ds.imgid2sg(img_id)
#     l_test_terms.append(sg['obj_labels'])    

'''f30k and coco'''
l_train_terms = []
for img_id in ds.d_split['train']:
    sg = ds.imgid2sg(img_id)
    l_row = []
    for i, word in enumerate(sg['node_labels']):
        if i < len(sg['bboxes']):
            l_row.append(word)
    l_train_terms.append(l_row)
    
l_test_terms = []
for img_id in ds.d_split['test']:
    sg = ds.imgid2sg(img_id)
    l_row = []
    for i, word in enumerate(sg['node_labels']):
        if i < len(sg['bboxes']):
            l_row.append(word)
    l_test_terms.append(l_row)

In [20]:
l_test = ds.d_split['test']

In [7]:
train_vocab = set([t for d in l_train_terms for t in d])

In [8]:
test_vocab = set([t for d in l_test_terms for t in d])

In [9]:
vocab = train_vocab.union(test_vocab)
vocab2idx = {v:i for i, v in enumerate(sorted(list(vocab)))}

In [10]:
train_mat = np.zeros((len(l_train_terms), len(vocab)))
test_mat = np.zeros((len(l_test_terms), len(vocab)))

In [11]:
len(train_vocab)

150

In [12]:
len(test_vocab)

149

In [13]:
for i_d, d in enumerate(l_train_terms):
    for t in d:
        idx = vocab2idx[t]
        train_mat[i_d, idx] += 1
        
for i_d, d in enumerate(l_test_terms):
    for t in d:
        idx = vocab2idx[t]
        test_mat[i_d, idx] += 1        

In [14]:
tfidf = TfidfTransformer(use_idf=True)

In [15]:
tfidf.fit(train_mat)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [16]:
test_tfidf = tfidf.transform(test_mat).toarray()

In [17]:
sim_mat = test_tfidf.dot(test_tfidf.T)

In [18]:
# os.mkdir('/data/project/rw/viewer_CBIR/viewer/vg_coco_results/boo_tf')
# os.mkdir('/data/project/rw/viewer_CBIR/viewer/f30k_results/boo_tfidf')
os.mkdir('/data/project/rw/viewer_CBIR/viewer/coco_results/boo_tfidf')

In [21]:
result_dir = '/data/project/rw/viewer_CBIR/viewer/coco_results/boo_tfidf'
for i_query, test_id in enumerate(tqdm(l_test)):  # query id
    l_sim = []
    for j_target, target_id in enumerate(l_test):
        l_sim.append(sim_mat[i_query, j_target])
    df = pd.DataFrame({'target_id': l_test, 'sim': l_sim}).sort_values('target_id')

    df[['target_id', 'sim']].to_csv(os.path.join(result_dir, f'{test_id}.tsv'), sep='\t', header=False, index=False)

100%|██████████| 5000/5000 [02:28<00:00, 33.68it/s]
